<a href="https://colab.research.google.com/github/MarcelodeFreitas/udemy_deep_learning_pytorch_python/blob/main/Tarefa_Iris_Salvar_e_carregar_o_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa: Iris - Salvar e carregar o classificador

## Etapa 1: Importação das bibliotecas

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim
from torch.nn import functional as F
torch.__version__

'1.4.0'

## Etapa 2: Base de dados

In [ ]:
torch.manual_seed(123)

In [ ]:
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [ ]:
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [ ]:
previsores = torch.tensor(previsores, dtype = torch.float)
classe = torch.tensor(classe, dtype = torch.long)

In [ ]:
dataset = torch.utils.data.TensorDataset(previsores, classe)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle = True)

## Etapa 3: Construção do modelo

In [ ]:
classificador = nn.Sequential(
                nn.Linear(4, 8),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(8, 8),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(8, 3),
                )

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classificador.parameters(), lr = 0.001, weight_decay = 0.0001)

## Etapa 4: Treinamento do modelo

In [ ]:
for epoch in range(2000):

    running_loss = 0.
    running_accuracy = 0.

    for data in train_loader:
        inputs, labels = data

        optimizer.zero_grad()

        outputs = classificador(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        ps = F.softmax(outputs)

        top_p, top_class = ps.topk(k = 1, dim = 1)

        equals = top_class == labels.view(*top_class.shape)

        running_accuracy += torch.mean(equals.type(torch.float))

    print('Época {:3d}: perda {:3.5f} - precisão {:3.5f}'.format(epoch + 1, running_loss/len(train_loader), running_accuracy/len(train_loader)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Época   1: perda 1.17812 - precisão 0.32000
Época   2: perda 1.13584 - precisão 0.34000
Época   3: perda 1.12890 - precisão 0.27333
Época   4: perda 1.11238 - precisão 0.18667
Época   5: perda 1.10700 - precisão 0.18667
Época   6: perda 1.09803 - precisão 0.36667
Época   7: perda 1.11251 - precisão 0.28000
Época   8: perda 1.08343 - precisão 0.34667
Época   9: perda 1.07394 - precisão 0.36667
Época  10: perda 1.07578 - precisão 0.35333
Época  11: perda 1.09621 - precisão 0.27333
Época  12: perda 1.08456 - precisão 0.29333
Época  13: perda 1.08200 - precisão 0.29333
Época  14: perda 1.08009 - precisão 0.30667
Época  15: perda 1.08110 - precisão 0.31333
Época  16: perda 1.06296 - precisão 0.36000
Época  17: perda 1.07327 - precisão 0.30667
Época  18: perda 1.05835 - precisão 0.33333
Época  19: perda 1.06602 - precisão 0.29333
Época  20: perda 1.05315 - precisão 0.34000
Época  21: perda 1.03839 - precisão 0.38000
Época  22: perda 1.04870 - precisão 0.30000
Época  23: perda 1.01084 - preci

## Etapa 5: Salvar e carregar o classificador e classificar um registro

In [ ]:
torch.save(classificador.state_dict(), 'checkpoint.pth')

In [ ]:
state_dict = torch.load('checkpoint.pth')
classificador.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
novo = torch.tensor([[3.2, 4.5, 0.9, 1.1]], requires_grad = False)
classificador.eval()
previsao = classificador(novo)
previsao = F.softmax(previsao)
previsao = (previsao > 0.5).numpy()
if previsao[0][0] == True and previsao[0][1] == False and previsao[0][2] == False:
    print('Iris setosa')
elif previsao[0][0] == False and previsao[0][1] == True and previsao[0][2] == False:
    print('Iris virginica')
elif previsao[0][0] == False and previsao[0][1] == False and previsao[0][2] == True:
    print('Iris versicolor')

Iris setosa


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.
